In [1]:
## importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
loan = pd.read_csv('loan.csv')

/Applications/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
loan.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [4]:
loan.shape

(39717, 111)

In [5]:
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(74), int64(13), object(24)
memory usage: 33.6+ MB


## Data Cleaning

### 1. Checking for Missing Values

The dataset has 111 columns. We will ignore the columns that have all values as null as they will not be helpful for our analysis

In [6]:
# columns that have all values as null
for x in loan.columns:
    if loan[x].isnull().sum()>=39717:
        print(x)

mths_since_last_major_derog
annual_inc_joint
dti_joint
verification_status_joint
tot_coll_amt
tot_cur_bal
open_acc_6m
open_il_6m
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
total_cu_tl
inq_last_12m
acc_open_past_24mths
avg_cur_bal
bc_open_to_buy
bc_util
mo_sin_old_il_acct
mo_sin_old_rev_tl_op
mo_sin_rcnt_rev_tl_op
mo_sin_rcnt_tl
mort_acc
mths_since_recent_bc
mths_since_recent_bc_dlq
mths_since_recent_inq
mths_since_recent_revol_delinq
num_accts_ever_120_pd
num_actv_bc_tl
num_actv_rev_tl
num_bc_sats
num_bc_tl
num_il_tl
num_op_rev_tl
num_rev_accts
num_rev_tl_bal_gt_0
num_sats
num_tl_120dpd_2m
num_tl_30dpd
num_tl_90g_dpd_24m
num_tl_op_past_12m
pct_tl_nvr_dlq
percent_bc_gt_75
tot_hi_cred_lim
total_bal_ex_mort
total_bc_limit
total_il_high_credit_limit


In [7]:
columns_notnull = [x for x in loan.columns if loan[x].isnull().sum()<39717]
len(columns_notnull) # columns that do not have all values as null

57

Checking if there are columns that have only one value across all the rows.
These rows are also not useful as they dont indicate any significant information

In [8]:
[x for x in columns_notnull if len(loan[x].value_counts())<=1]

['pymnt_plan',
 'initial_list_status',
 'collections_12_mths_ex_med',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'tax_liens']

In [9]:
#Checking the unique values for each column that has only one unique value
for x in columns_notnull:
    if len(loan[x].value_counts())<=1:
        print('Unique Values in {} are {}'.format(x,loan[x].value_counts().index[0]))

Unique Values in pymnt_plan are n
Unique Values in initial_list_status are f
Unique Values in collections_12_mths_ex_med are 0.0
Unique Values in policy_code are 1
Unique Values in application_type are INDIVIDUAL
Unique Values in acc_now_delinq are 0
Unique Values in chargeoff_within_12_mths are 0.0
Unique Values in delinq_amnt are 0
Unique Values in tax_liens are 0.0


We can safely ignore these columns too for our analysis

In [10]:
# removing the columns that have same value
columns_notnull = [x for x in columns_notnull if len(loan[x].value_counts())>1]
len(columns_notnull)

48

In [11]:
for x in columns_notnull:
    if loan[x].isnull().sum()>0:
        print(x, loan[x].isnull().sum())

emp_title 2458
desc 12939
title 10
mths_since_last_delinq 25682
mths_since_last_record 36931
revol_util 50
last_pymnt_d 71
next_pymnt_d 38577
last_credit_pull_d 2
pub_rec_bankruptcies 697


Now that we have 48 columns, There are 7 columns that have null values. We will review the columns that have null values one by one

In [12]:
for x in columns_notnull:
    if loan[x].isnull().sum()>0:
        print(x, loan[x].isnull().sum())

emp_title 2458
desc 12939
title 10
mths_since_last_delinq 25682
mths_since_last_record 36931
revol_util 50
last_pymnt_d 71
next_pymnt_d 38577
last_credit_pull_d 2
pub_rec_bankruptcies 697


#### emp_title : "The job title supplied by the Borrower when applying for the loan."

1. emp_title has 2458 null values
2. Also in the data dictionary it is mentioned that "Employer Title replaces Employer Name for all loans listed after 9/23/2013" 

But there is no column indicating employer name, hence we will add all null values under a category 'Other'

In [13]:
#Checking if 'Other' value is already present
loan[loan.emp_title.str.lower()=='other'].emp_title

23591    Other
25498    other
Name: emp_title, dtype: object

In [14]:
#filling all null values as 'Other'
loan.emp_title.fillna('Other', inplace=True)
loan.emp_title.isnull().sum() # checking if all null values are filled

0

#### desc : "loan description provided by the borrower"

1. desc column has 12939 null values. 
2. We can fill all the null values with 'Not Provided'

In [15]:
loan.desc[3]

"  Borrower added on 12/21/11 > to pay for property tax (borrow from friend, need to pay back) & central A/C need to be replace. I'm very sorry to let my loan expired last time.<br>"

In [16]:
loan.desc.fillna('Not Provided', inplace=True)
loan.desc.isnull().sum() # checking if all null values are filled

0

#### title : "The loan title provided by the borrower"

1. title has 10 null values
2. We can fill the null values as 'Not Provided'

In [17]:
loan.title[0:10]

0                                 Computer
1                                     bike
2                     real estate business
3                                 personel
4                                 Personal
5    My wedding loan I promise to pay back
6                                     Loan
7                          Car Downpayment
8     Expand Business & Buy Debt Portfolio
9              Building my credit history.
Name: title, dtype: object

In [18]:
loan.title.fillna('Not Provided', inplace=True)
loan.title.isnull().sum()

0

#### mths_since_last_delinq : "The number of months since the borrower's last delinquency."

1. mths_since_last_delinq has 25682 null values
2. As per the description of the field, this column indicates the number of months if borrower is delinquent

In [19]:
print ('Percentage of null values present = {}%. There are {} null values in this field out of {} values'
       .format(round(100* loan[loan.mths_since_last_delinq.isnull()].shape[0]/loan.shape[0],2), 
               loan[loan.mths_since_last_delinq.isnull()].shape[0], 
               loan.shape[0])
      )

Percentage of null values present = 64.66%. There are 25682 null values in this field out of 39717 values


Hence removing this column from our analysis

In [20]:
columns_notnull.remove('mths_since_last_delinq')

#### mths_since_last_record : "The number of months since the last public record."

1. mths_since_last_record has 36931 null values

Lets look at the percentage of null values

In [21]:
print ('Percentage of null values present = {}%. There are {} null values in this field out of {} values'
       .format(round(100* loan[loan.mths_since_last_record.isnull()].shape[0]/loan.shape[0],2), 
               loan[loan.mths_since_last_record.isnull()].shape[0], 
               loan.shape[0])
      )

Percentage of null values present = 92.99%. There are 36931 null values in this field out of 39717 values


Hence removing this column from our analysis

In [22]:
columns_notnull.remove('mths_since_last_record')

#### revol_util : "Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit."

1. There are 50 null values in revol_util column
2. This column indicates the revolving line utilization rate.
3. We can replace all the null values to 0% (min value) so that we can utilize the not-null values during our analysis

In [23]:
print ('Percentage of null values present = {}%. There are {} null values in this field out of {} values'
       .format(round(100* loan[loan.revol_util.isnull()].shape[0]/loan.shape[0],2), 
               loan[loan.revol_util.isnull()].shape[0], 
               loan.shape[0])
      )

Percentage of null values present = 0.13%. There are 50 null values in this field out of 39717 values


In [24]:
loan.revol_util.fillna('0',inplace=True)
loan.revol_util.isnull().sum() # checking if the null values are replaced

0

#### last_pymnt_d : "Last month payment was received"

1. This column has 71 null values

In [25]:
loan[loan.last_pymnt_d.isnull()>0].last_pymnt_amnt.value_counts()

0.0    71
Name: last_pymnt_amnt, dtype: int64

In [26]:
loan.last_pymnt_d.value_counts()

May-16    1256
Mar-13    1026
Dec-14     945
May-13     907
Feb-13     869
Apr-13     851
Mar-12     844
Jan-14     832
Aug-14     832
Aug-12     832
Oct-12     826
Mar-14     824
Jul-14     820
Oct-14     808
Feb-14     792
Jan-13     784
Dec-13     780
Jun-14     778
Jul-13     776
Sep-12     761
Jul-12     741
Nov-12     740
May-12     736
Feb-12     735
Apr-12     734
Aug-13     727
Dec-12     707
Jun-12     702
Sep-14     693
Oct-13     692
          ... 
Apr-15     136
May-10     116
Jun-10     116
Apr-10     114
Feb-10     112
Dec-09      93
Jan-10      73
Oct-09      62
Nov-09      59
Aug-09      49
Jul-09      43
Apr-09      43
May-09      41
Jun-09      40
Mar-09      38
Sep-09      37
Feb-09      33
Oct-08      28
Jan-09      21
May-08      14
Jul-08      14
Dec-08      14
Aug-08      13
Sep-08      12
Apr-08      12
Jun-08      10
Nov-08      10
Mar-08       5
Jan-08       4
Feb-08       1
Name: last_pymnt_d, dtype: int64

2. The blank values in this column indicates that last month payment was not received as the last_pymnt_amnt is null.
3. We decide to keep the null values as is for our further analysis

#### next_pymnt_d : "Next scheduled payment date"

1. next_pymnt_d has 38577 null values
2. Next payment date will be scheduled for loans that are not yet paid fully. So let us look at the loan status to compare this field

In [27]:
loan[loan.next_pymnt_d.isnull()>0].loan_status.value_counts()

Fully Paid     32950
Charged Off     5627
Name: loan_status, dtype: int64

all null values in next_pymnt_d fall under 'Fully Paid' or 'Charged Off' under loan_status column. we have to keep this column to analyze next_pymnt_d on other loan statuses

#### last_credit_pull_d : "The most recent month LC pulled credit for this loan"

1. last_credit_pull_d has 2 null values
2. Since the null values are only 2, We will keep this column for our analysis

In [28]:
loan.last_credit_pull_d.describe()

count      39715
unique       106
top       May-16
freq       10308
Name: last_credit_pull_d, dtype: object

#### pub_rec_bankruptcies : "Number of public record bankruptcies"

1. There are 697 null values in this field

In [29]:
loan.pub_rec_bankruptcies.describe()

count    39020.000000
mean         0.043260
std          0.204324
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          2.000000
Name: pub_rec_bankruptcies, dtype: float64

Since the minimum is 0 for this column and maximum is 2, Let us fill the null values with 0 (no records)

In [30]:
loan.pub_rec_bankruptcies.fillna(0,inplace=True)

Now the data is cleaned for missing values and all the necessary columns names are captured in the list columns_notnull

In [31]:
print('We have the below {} columns after cleaning the dataset for missing values \n \n {}'
      .format(len(columns_notnull),columns_notnull))

We have the below 46 columns after cleaning the dataset for missing values 
 
 ['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'pub_rec_bankruptcies']


### 2. Delete columns that are unnecessary

#### id: A unique LC assigned ID for the loan listing.

In [32]:
print("Number of unique values in 'id' column = {}".format(len(loan.id.unique())))

Number of unique values in 'id' column = 39717


In the loan dataset, the column 'id' is the unique identifier for the LC and all the values are unique. 
Let us remove the column for our analysis

In [33]:
columns_notnull.remove('id')

#### member_id: A unique LC assigned Id for the borrower member.

In [34]:
print("Number of unique values in 'member_id' column = {}".format(len(loan.member_id.unique())))

Number of unique values in 'member_id' column = 39717


Similar to the 'id' column, member_id also has all unique values. So let us remove the column for our analysis

In [35]:
columns_notnull.remove('member_id')

In [36]:
print("We have the below {} columns after 'deleting the identified columns' \n \n {}".format(len(columns_notnull),columns_notnull))

We have the below 44 columns after 'deleting the identified columns' 
 
 ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'pub_rec_bankruptcies']


In [37]:
# loan dataset is sliced to have only the above columns
loan = loan[columns_notnull]

In [38]:
loan.shape

(39717, 44)

### 3. Convert incorrect data types:

There are date columns in the dataset that needs to be corrected to proper datetime format

['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']

#### issue_d

In [44]:
loan.issue_d.value_counts()[0:10]

Dec-11    2260
Nov-11    2223
Oct-11    2114
Sep-11    2063
Aug-11    1928
Jul-11    1870
Jun-11    1827
May-11    1689
Apr-11    1562
Mar-11    1443
Name: issue_d, dtype: int64

The issue_d has only month and year and not the date of the month.

In [45]:
loan.last_pymnt_d.value_counts()[0:10]

May-16    1256
Mar-13    1026
Dec-14     945
May-13     907
Feb-13     869
Apr-13     851
Mar-12     844
Jan-14     832
Aug-14     832
Aug-12     832
Name: last_pymnt_d, dtype: int64

In [46]:
loan.next_pymnt_d.value_counts()[0:10]

Jun-16    1125
Jul-16      15
Name: next_pymnt_d, dtype: int64

In [47]:
loan.last_credit_pull_d.value_counts()[0:10]

May-16    10308
Apr-16     2547
Mar-16     1123
Feb-13      843
Feb-16      736
Jan-16      657
Dec-15      647
Mar-13      577
Mar-14      564
Dec-14      562
Name: last_credit_pull_d, dtype: int64

Similar to issue_d, all the above dates contain only month and Year. 